In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import transforms, utils
from torchvision.transforms import Resize, ToTensor, Normalize, RandomVerticalFlip
from torch.utils.data import Dataset, DataLoader, random_split, SubsetRandomSampler, WeightedRandomSampler
from torch.optim.lr_scheduler import CosineAnnealingLR

from PIL import Image
from tqdm.auto import tqdm

from sklearn.model_selection import KFold
from efficientnet_pytorch import EfficientNet

In [2]:
TRAIN_PATH = '/opt/ml/input/data/train/'
TRAIN_IMAGE_PATH = '/opt/ml/input/data/train/processed_train_images/'

In [3]:
train_df = pd.read_csv(os.path.join(TRAIN_PATH, 'preprocessed_train.csv'))
train_df.sample(10)

,Unnamed: 0,id,gender,mask,age,age_3,age_11,label,path
12154,12154,001644,0,0,22,0,2,0,/opt/ml/input/data/train/processed_train_image...
9856,9856,006236,0,0,20,0,2,0,/opt/ml/input/data/train/processed_train_image...
2653,2653,003290,1,0,19,0,1,3,/opt/ml/input/data/train/processed_train_image...
2456,2456,005517,1,0,56,1,5,4,/opt/ml/input/data/train/processed_train_image...
13314,13314,001087,1,1,25,0,2,9,/opt/ml/input/data/train/processed_train_image...
8061,8061,000675,1,0,59,1,5,4,/opt/ml/input/data/train/processed_train_image...
4319,4319,000678,1,0,52,1,5,4,/opt/ml/input/data/train/processed_train_image...
3493,3493,003006,1,0,19,0,1,3,/opt/ml/input/data/train/processed_train_image...
5207,5207,005092,0,0,52,1,5,1,/opt/ml/input/data/train/processed_train_image...
6353,6353,005440,1,2,50,1,5,16,/opt/ml/input/data/train/processed_train_image...


In [4]:
class maskDataset(Dataset):
    def __init__(self, train_df, transform, train=True):
        self.train = train
        self.transform = transform

        if self.train == True:
            self.paths = train_df['path']
            self.labels = train_df['age_3']
        else:
            self.paths = train_df
        

    def __getitem__(self, index):
        if self.train:
            image = Image.open(self.paths.iloc[index])
            label = self.labels.iloc[index]
        else:
            image = Image.open(self.paths[index])

        if self.transform:
            image = self.transform(image)

        if self.train:
            return image, torch.tensor(label)
        else:
            return image

    def __len__(self):
        return len(self.paths)

In [5]:
transform = transforms.Compose([
    # Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    # Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2))
    RandomVerticalFlip()
])

In [6]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(train_df, test_size=0.2, shuffle=True, stratify=train_df['label'], random_state=42)

In [7]:
train.shape, valid.shape

((15114, 9), (3779, 9))

In [8]:
BATCH_SIZE = 32

In [9]:
train_dataset = maskDataset(train, transform)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=1)

In [10]:
valid_dataset = maskDataset(valid, transform)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True, num_workers=1)

In [11]:
next(iter(train_dataloader))[0].shape

torch.Size([32, 3, 512, 384])

In [12]:
class EfficientNet_MultiLabel(nn.Module):
    def __init__(self, in_channels=3, num_classes=3):
        super(EfficientNet_MultiLabel, self).__init__()
        self.in_channels = in_channels
        self.num_classes = num_classes
        self.network = EfficientNet.from_pretrained('efficientnet-b0', in_channels=self.in_channels, num_classes=self.num_classes)

    def forward(self, x):
        return self.network(x)

efficientNet = EfficientNet_MultiLabel()

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ('My Device :', device)
efficientNet = efficientNet.to(device)

Loaded pretrained weights for efficientnet-b0
My Device : cuda:0


In [13]:
LEARNING_RATE = 1e-4
NUM_EPOCH = 10

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(efficientNet.parameters(), lr=LEARNING_RATE)

dataloaders = {
    "train" : train_dataloader,
    "valid" : valid_dataloader
}

In [15]:
scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=20)

In [16]:
best_acc = 0
best_loss = 0

PRINT_EVERY = 10

for epoch in tqdm(range(NUM_EPOCH)):
    for phase in ['train', 'valid']:
        running_loss = 0
        running_acc = 0

        if phase == 'train':
            efficientNet.train()
        elif phase == 'valid':
            efficientNet.eval()

        for idx, (images, labels) in enumerate(dataloaders[phase]):
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                logits = efficientNet(images)
                _, pred = torch.max(logits, 1)
                loss = loss_fn(logits, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * images.size(0)
            running_acc += torch.sum(pred == labels.data)

            # if idx % PRINT_EVERY == 0:
            #     train_loss = running_loss / PRINT_EVERY
            #     train_acc = running_acc / BATCH_SIZE / PRINT_EVERY
            #     print (f"Epoch={epoch}, Training Loss={train_loss:4.4}, Training Accuracy={train_acc:4.2%}, lr={LEARNING_RATE}")
            
        scheduler.step()

        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_acc / len(dataloaders[phase].dataset)
        print (f'[{phase}] 현재 EPOCH : {epoch}, loss : {epoch_loss}, acc : {epoch_acc}')


KeyboardInterrupt: 